In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype

In [ ]:
from svgpathtools import svg2paths


# try 1

In [ ]:
svgpath = '/mnt/c/code/side/plotter_images/u2_visor_portrait_trace.svg'
doc = vpype.read_svg(svgpath, quantization=0.1, simplify=True, crop=True, parallel=True)

In [ ]:
paths, attributes = svg2paths(svgpath)

In [ ]:
def complex_to_xy(point):
    return point.real, point.imag

In [ ]:
def svgLine_to_LineString(line):
    return LineString([
        complex_to_xy(line.start),
        complex_to_xy(line.end),
    ])

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 14 # inches
border:float = 20.

buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)
xmin, ymin, xmax, ymax = drawbox.bounds

In [ ]:
mls = so.linemerge(MultiLineString([svgLine_to_LineString(line) for line in paths[1]]))
fills = MultiPolygon([Polygon(ls) for ls in mls])

lines = so.linemerge(MultiLineString([svgLine_to_LineString(line) for line in paths[0]]))

In [ ]:
sg.GeometryCollection([fills, lines])

In [ ]:
def consolidate_polygons2(polys):
    '''For if polygons are inside each other '''
    consolidated_polys = []
    for p0 in polys:
        other_polys = [p for p in polys if p != p0]
        p0_contains = []
        p0_is_within = []
        for p1 in other_polys:
            if p0.contains(p1):
                p0_contains.append(p1)
            elif p0.within(p1):
                p0_is_within.append(p1)
        
        new_p = Polygon(p0)
        if not any(p0_is_within):
            new_p = gp.robust_difference(p0, so.unary_union(p0_contains))
            consolidated_polys.append(new_p)
    return consolidated_polys

In [ ]:
mp = MultiPolygon([p for p in fills if p.area > 0])

In [ ]:
sns.displot([np.log10(p.area) for p in fills])

In [ ]:
small_polys = []
large_polys = []
thresh = 0.1
for p in mp:
    if p.area > thresh:
        large_polys.append(p.buffer(1e-6))
    else:
        small_polys.append(p)

In [ ]:
gp.merge_Polygons(large_polys)

In [ ]:
lps = list(gp.merge_Polygons(large_polys))

In [ ]:
areas = [p.area for p in lps]

In [ ]:
sort_lps = [lps[i] for i in np.argsort(areas)]
consolidated_ps = consolidate_polygons2(sort_lps)
MultiPolygon(consolidated_ps)

In [ ]:
all_polys = MultiPolygon(lps + small_polys)

In [ ]:
scaled_geos = gp.make_like(sg.GeometryCollection([lines, all_polys]), drawbox)

In [ ]:
traces = scaled_geos[0]
polys = scaled_geos[1]

In [ ]:

sapb = polys.buffer(1e-3)
sapb

In [ ]:
stp = gp.ScaleTransPrms(n_iters=100, d_buffer=-0.2, d_translate_factor=0.3, angles=np.radians(-90))
# stp.d_buffers *= gp.gaussian_random_walk(n=stp.d_buffers.shape[0], step_std=10)

In [ ]:
all_fills = []
for p in sapb:
    fills = gp.scale_trans(p, **stp.prms)
    mfills = gp.merge_LineStrings([f.boundary for f in fills])
    all_fills.append(mfills)

In [ ]:
fill_layer = gp.merge_LineStrings(all_fills)

In [ ]:
min_path_length = 5
filter_fill_layer = MultiLineString([f for f in fill_layer if f.length>min_path_length])

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(traces)
sk.geometry(filter_fill_layer)
sk.geometry(traces.buffer(1, cap_style=2, join_style=2).boundary)
sk.display(color_mode='None')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0110_me_trace.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.05mm linemerge --tolerance 0.3mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)